In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
print(list_city_dt)
print(len(list_city_dt))

['paris_db=2.00', 'paris_db=0.50', 'paris_db=1.00', 'lyon_db=2.00', 'frankfurt_db=0.50', 'berlin_db=2.00', 'lyon_db=0.50', 'frankfurt_db=1.00', 'berlin_db=0.50', 'frankfurt_db=2.00', 'lyon_db=1.00', 'berlin_db=4.00', 'berlin_db=1.00', 'paris_db=4.00', 'lyon_db=4.00', 'frankfurt_db=4.00']
16


In [13]:
list_results = []
dict_workforce = {}
dict_workforce['instance'] = []
dict_workforce['OC'] = []
dict_workforce['model'] = []
dict_workforce['max_n_shifts'] = []
dict_workforce['workforce_size'] = []
dict_workforce['objective_value'] = []
dict_workforce['workforce_size_next'] = []
dict_workforce['objective_value_next'] = []

MAX_TRIES = 3

RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3


# for base_file in list_city_dt:
for base_file in ['berlin_db=1.00']:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"
    
    # for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    for OC in [1.5]:
        # for model in ['fixed', 'flex', 'partflex']:
        for model in ['partflex']:
            if model == 'partflex':
                # for max_n_shifts in range(2,5):
                for max_n_shifts in [2]:
                    dict_base = {
                        'instance_file':[file],
                        'city':[city],
                        'demand_baseline':[demand_baseline],
                        'model':[model]*n_regions,
                        'max_n_shifts':[max_n_shift]*n_regions,
                        'outsourcing_cost_multiplier':[OC]*n_regions,
                        'regional_multiplier':[RM]*n_regions,
                        'global_multiplier':[GM]*n_regions,
                        'region':[region for region in range(0,n_regions)]
                    }


                    objvalprev = 10000000
                    objval = 1000000
                    for trial_size in range(1, MAX_TRIES):
                        workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                        if (objval < objvalprev)&(np.isnan(objval) == False):
                            objvalprev = objval
                            dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff, max_n_shifts)
                            objval = dict_results['objective_value'][0]
                            df_ = pd.DataFrame(dict_results)
                            df_['workforce_size_trial'] = trial_size
                            list_results.append(df_)
                        else:
                            #keep track of optimal
                            dict_workforce['instance'].append(base_file)
                            dict_workforce['OC'].append(OC)
                            dict_workforce['model'].append(model)
                            dict_workforce['max_n_shifts'].append(np.nan)
                            dict_workforce['workforce_size'].append(trial_size-1)
                            dict_workforce['objective_value'].append(objvalprev)
                            dict_workforce['objective_value_next'].append(objval)
                            trial_size = 10000000000
                        if trial_size == int(MAX_TRIES-1):
                            #keep track of optimal
                            dict_workforce['instance'].append(base_file)
                            dict_workforce['OC'].append(OC)
                            dict_workforce['model'].append(model)
                            dict_workforce['max_n_shifts'].append(np.nan)
                            dict_workforce['workforce_size'].append(trial_size)
                            dict_workforce['objective_value'].append(objvalprev)
                            dict_workforce['objective_value_next'].append(objval)
            else:
                objvalprev = 10000000
                objval = 1000000
                for trial_size in range(1, MAX_TRIES):
                    print("trial_size",trial_size)
                    workforce_dict = {0:trial_size, 1:1, 2:1, 3:1, 4:1}
                    if (objval < objvalprev)&(np.isnan(objval) == False):
                        objvalprev = objval
                        dict_results = run_roster_solver_objval(model, instance_file_weekday, shift_file_weekday, instance_file_weekend, shift_file_weekend, workforce_dict, OC, RM, GM, h_min, h_max, max_n_diff)
                        objval = dict_results['objective_value'][0]
                        print("objective_value",objval)
                        df_ = pd.DataFrame(dict_results)
                        df_['workforce_size_trial'] = trial_size
                        list_results.append(df_)
                    else: 
                        #keep track of optimal (trial size - 1)
                        dict_workforce['instance'].append(base_file)
                        dict_workforce['OC'].append(OC)
                        dict_workforce['model'].append(model)
                        dict_workforce['max_n_shifts'].append(np.nan)
                        dict_workforce['workforce_size'].append(trial_size-1)
                        dict_workforce['objective_value'].append(objvalprev)
                        dict_workforce['objective_value_next'].append(objval)
                        trial_size = 10000000000
                    if trial_size == int(MAX_TRIES-1):
                        #keep track of optimal
                        dict_workforce['instance'].append(base_file)
                        dict_workforce['OC'].append(OC)
                        dict_workforce['model'].append(model)
                        dict_workforce['max_n_shifts'].append(np.nan)
                        dict_workforce['workforce_size'].append(trial_size)
                        dict_workforce['objective_value'].append(objvalprev)
                        dict_workforce['objective_value_next'].append(objval)
                        trial_size = 10000000000


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 99524 rows, 102471 columns and 156227 nonzeros
Model fingerprint: 0x5c6f2a5c
Variable types: 99120 continuous, 3351 integer (3351 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [3e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 99180 rows and 99127 columns
Presolve time: 0.03s
Presolved: 344 rows, 3344 columns, 12548 nonzeros
Variable types: 0 continuous, 3344 integer (3344 binary)
Found heuristic solution: objective 9460.3166667

Explored 1 nodes (0 simplex iterations) in 0.07 seconds (0.05 work units)
Thread count was 8 (of 8 avai

In [14]:
dict_workforce

{'instance': ['berlin_db=1.00'],
 'OC': [1.5],
 'model': ['partflex'],
 'max_n_shifts': [nan],
 'workforce_size': [2],
 'objective_value': [9460.316666666355],
 'workforce_size_next': [],
 'objective_value_next': [9384.13333333302]}

In [ ]:

dict_workforce['instance'] = []
dict_workforce['OC'] = []
dict_workforce['model'] = []
dict_workforce['max_n_shifts'] = []
dict_workforce['workforce_size'] = []
dict_workforce['objective_value'] = []
dict_workforce['objective_value_next'] = []


for file in os.listdir(r'../instances'):
    if (file.find('doublepeak') > -1) or (file.find('uniform') > -1):
        if file not in os.listdir(r'../workforce_size'):

for base_file in ['berlin_db=1.00']:
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"


    workforce_out = f"../workforce_size/{base_file}"
    dict_out = {
        'instance_file':[],
        'city':[],
        'demand_baseline':[],
        'demand_type':[],
        'model':[],
        'max_n_shifts':[],
        'outsourcing_cost_multiplier':[],
        'regional_multiplier':[],
        'global_multiplier':[],
        'workforce_size':[],
        'objective_value':[],
        'objective_value_next':[]
    }

    city_pattern = r'(\w+)_db'
    db_pattern = r'db=(\d+\.\d+)'
    dt_pattern = r'dt=(\w+)'

    city_match = re.search(city_pattern, file)
    db_match = re.search(db_pattern, file)
    dt_match = re.search(dt_pattern, file)
    
    city = city_match.group(1) if city_match else None
    demand_baseline = float(db_match.group(1)) if db_match else None













            for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
                for RM in [1.5]:
                    for GM in [0.8]:
                        for model in ['fixed','flex','partflex']:
                            if model == 'partflex':
                                for max_n_shift in range(2,5):
                                    dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM, max_n_shifts=max_n_shift)
                                    # if max_n_shift == 2:
                                    #     dict_raw_out = dict_raw.copy()
                                    dict_base = {
                                        'instance_file':[file]*n_regions,
                                        'city':[city]*n_regions,
                                        'demand_baseline':[demand_baseline]*n_regions,
                                        'demand_type':[demand_type]*n_regions,
                                        'model':[model]*n_regions,
                                        'max_n_shifts':[max_n_shift]*n_regions,
                                        'outsourcing_cost_multiplier':[OC]*n_regions,
                                        'regional_multiplier':[RM]*n_regions,
                                        'global_multiplier':[GM]*n_regions,
                                        'region':[region for region in range(0,n_regions)]
                                    }
                                    list_shift_start = []
                                    list_shift_end = []
                                    for region in range(0, n_regions):
                                        if region in dict_shifts.keys():
                                            list_shift_start.append(dict_shifts[region]['shifts_start'])
                                            list_shift_end.append(dict_shifts[region]['shifts_end'])
                                        else:
                                            list_shift_start.append({})
                                            list_shift_end.append({})
                                    dict_base['shifts_start'] = list_shift_start
                                    dict_base['shifts_end'] = list_shift_end
                                    for key in dict_base.keys():
                                        dict_out[key].extend(dict_base[key])
                            else:
                                dict_shifts, n_regions, dict_raw = run_solver_shift_return(model=model, instance=instance_, outsourcing_cost_multiplier=OC, regional_multiplier=RM, global_multiplier=GM)
                                dict_base = {
                                    'instance_file':[file]*n_regions,
                                    'city':[city]*n_regions,
                                    'demand_baseline':[demand_baseline]*n_regions,
                                    'demand_type':[demand_type]*n_regions,
                                    'model':[model]*n_regions,
                                    'max_n_shifts':[np.nan]*n_regions,
                                    'outsourcing_cost_multiplier':[OC]*n_regions,
                                    'regional_multiplier':[RM]*n_regions,
                                    'global_multiplier':[GM]*n_regions,
                                    'region':[region for region in range(0,n_regions)]
                                }
                                list_shift_start = []
                                list_shift_end = []
                                for region in range(0, n_regions):
                                    if region in dict_shifts.keys():
                                        list_shift_start.append(dict_shifts[region]['shifts_start'])
                                        list_shift_end.append(dict_shifts[region]['shifts_end'])
                                    else:
                                        list_shift_start.append({})
                                        list_shift_end.append({})
                                dict_base['shifts_start'] = list_shift_start
                                dict_base['shifts_end'] = list_shift_end
                                for key in dict_base.keys():
                                    dict_out[key].extend(dict_base[key])

            with open(shift_out, 'w') as f:
                json.dump(dict_out, f, indent=2)

In [ ]:
            shift_out = f"../shifts/{file}"
            dict_out = {
                'instance_file':[],
                'city':[],
                'demand_baseline':[],
                'demand_type':[],
                'model':[],
                'max_n_shifts':[],
                'outsourcing_cost_multiplier':[],
                'regional_multiplier':[],
                'global_multiplier':[],
                'region':[],
                'shifts_start':[],
                'shifts_end':[]
            }